In [ ]:
from random import shuffle 
import cv2
import uuid
from tqdm import tqdm

import os
import pydicom
import numpy as np

In [ ]:
# dir Setup

data_dir = "/media/mic/ML/Training_Data/DEC-17-17/train/GRADE_2/"
output_dir = "/media/mic/ML/Training_Data/DEC-17-17/train/grade_2/"
patients = os.listdir(data_dir)

In [ ]:
print(patients)
print("")
print(str("Patients - ") + str(len(patients)))

In [ ]:
def process_training_data():

    # Define Vars
    PX_SIZE = 256
    IMAGES_FOUND = 0
    MAX_IMAGES_COUNT = 10000

    training_data = []
    label = [0,1,0,0]
    
    
    # Patients by ID
    for patient_id in patients:

        
        for user in tqdm(os.listdir(data_dir + patient_id)):
            patient_path = data_dir + patient_id + '/' + user

            patient_scans = os.listdir(patient_path)

            
            # Patient each Scans
            for patient_scan in patient_scans:
                data_path = patient_path + '/' + patient_scan

                # Scan Images .dcm
                for s in os.listdir(data_path):
    
                    if IMAGES_FOUND <= MAX_IMAGES_COUNT:
                        slice = pydicom.read_file(data_path + '/' + s)
                
                        if slice.pixel_array.shape[0] >= PX_SIZE and slice.pixel_array.shape[1] >= PX_SIZE:
                            # Resize
                            img = cv2.resize(np.array(slice.pixel_array),(PX_SIZE,PX_SIZE))
                    
                            # Save As PNG
                            #cv2.imwrite( str(output_dir) + str(uuid.uuid1()) + ".jpg", img)
                        
                            # Add to Array
                            training_data.append([np.array(img),np.array(label)])
                            IMAGES_FOUND += 1
                        else:
                            print("Low Resolution Image", slice.pixel_array.shape)

                    elif IMAGES_FOUND > MAX_IMAGES_COUNT:
                        return training_data

                    if IMAGES_FOUND % 100 == 0:
                        print(IMAGES_FOUND)
                        shuffle(training_data)
                        np.save('train_data_256X256_grade_2.npy', training_data)
    
    return training_data

In [ ]:
training_data = process_training_data()